In [ ]:
import pandas as pd


In [ ]:
url_datos_csv = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vSfilaqDNJez3zoMqcok02nDBjZhfEwDA5PXBWfx5fOj-QJSk1C4Wb1aoQ4q0gPgQEPzgfq5tUpgsif/pub?gid=228759331&single=true&output=csv'

df = pd.read_csv(url_datos_csv)

In [ ]:
df.sample(3 )

In [ ]:
 #Paleta de colores para los estados
paleta_colores = {
        'En ejecución': '#FFD700',  # Amarillo
        'Terminado-sin liquidar': '#FF0000',  # Rojo
        'Terminado-Liquidado': '#008000',  # Verde
        'Cerrado': '#008000',  # Verde
        'Desierto': '#800080',  # Morado
        'Firmado': '#0000FF',  # Azul
        'Sin Contrato': '#000000',  # Negro
        'Suspendido': '#808080'  # Gris
    }

def estado_a_boton_html(estado: str, tamano_fuente_tooltip= 12, colores=paleta_colores) -> str:
    colores = paleta_colores

    color = colores.get(estado, '#CCCCCC')  # Color por defecto si no se encuentra
    html = f'''
    <button style="
        background-color: {color};
        color: white;
        border: none;
        padding: 0px 0px;
        border-radius: 0px;
        font-size: {tamano_fuente_tooltip}px;
        cursor: default;
    ">
        {estado}
    </button>
    '''
    return html

In [ ]:
a=estado_a_boton_html('Firmado',tamano_fuente_tooltip= 12, colores=paleta_colores)
print(a)

In [ ]:
import streamlit as st


st.sidebar.title("Controles")
nombre = st.sidebar.text_input("Tu nombre")
edad = st.sidebar.slider("Edad", 0, 100, 25)
mostrar = st.sidebar.checkbox("Mostrar saludo")

if mostrar:
    st.write(f"Hola {nombre}, tienes {edad} años.")

In [ ]:
# MVB ================  Codigo final de pruebas ===============
# import streamlit as st
import geopandas as gpd
import folium
from streamlit_folium import st_folium

# Título
st.title("Mapa del Municipio de Piedecuesta")

# Cargar datos desde un archivo GeoJSON o SHP
#gdf = gpd.read_file("COL_adm2.geojson")  # GeoJSON con municipios de Colombia
gdf = gpd.read_file("gadm41_COL_2.json") 


# Filtrar solo Piedecuesta
piedecuesta = gdf[gdf['NAME_2'] == 'Piedecuesta']  # Asegúrate del nombre exacto

# Obtener el centro del mapa para enfocar
centro = piedecuesta.geometry.centroid.iloc[0]
mapa = folium.Map(location=[centro.y, centro.x], zoom_start=11)

# Agregar el polígono del municipio al mapa
folium.GeoJson(piedecuesta).add_to(mapa)

# Mostrar el mapa en Streamlit
st_folium(mapa, width=700, height=500)

In [ ]:
import geopandas as gpd
import folium

def generar_poligono_piedecuesta():
    #Dibujar el contorno del municipio de Piedecuesta

    # Cargar el archivo GeoJSON con todos los municipios de Colombia
    gdf = gpd.read_file("gadm41_COL_2.json")

    # Verifica cómo se llama la columna del municipio (puede ser NAME_2 o similar)
    print(gdf.columns)  # Solo la primera vez para revisar

    # Filtra el municipio de Piedecuesta
    piedecuesta = gdf[gdf['NAME_2'].str.lower() == 'piedecuesta']

    # Verifica que lo haya encontrado
    if piedecuesta.empty:
        print("No se encontró el municipio de Piedecuesta.")
        exit()

    # Obtener el centro del municipio para centrar el mapa
    centro = piedecuesta.geometry.centroid.iloc[0]
    lat, lon = centro.y, centro.x

    # Crear el mapa centrado en Piedecuesta
    mapa = folium.Map(location=[lat, lon], zoom_start=11)

    # Añadir el polígono del municipio al mapa
    folium.GeoJson(piedecuesta, name="Piedecuesta").add_to(mapa)
    #return folium.GeoJson(piedecuesta, name="Piedecuesta")

    return mapa

    # Guardar el mapa como archivo HTML
    #mapa.save("mapa_piedecuesta.html")
    #print("Mapa guardado como 'mapa_piedecuesta.html'")

## Generar poligono de piedecuesta

In [ ]:

generar_poligono_piedecuesta()
dibujar_piedecuesta()

## COORDENADAS DEL CSV A LATITUD Y LONGITUD DECIMAL
### Probar replicar las filas de un contrato cuando tiene mas de un par de coordenadas

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None) 
pd.options.display.max_rows = None


url_datos_csv = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vSfilaqDNJez3zoMqcok02nDBjZhfEwDA5PXBWfx5fOj-QJSk1C4Wb1aoQ4q0gPgQEPzgfq5tUpgsif/pub?gid=228759331&single=true&output=csv'
#Codificacion del archivo CSV. Si hay problemas con acentos, probar 'latin-1'
encoding='utf-8' 

df = pd.read_csv(url_datos_csv, encoding=encoding)
df["size"]=df.Coordenadas.str.len()
df = df[["NUMERO DE PROCESO", "Coordenadas","size"]]
df.shape
#df.to_csv("df_importado.csv")


In [ ]:
 df.sort_values(by='size', ascending=False).Coordenadas.unique().tolist()
#df.Coordenadas.unique()#.tolist()


In [ ]:
df["size"]=df.Coordenadas.str.len()

In [ ]:

pd.set_option('display.max_columns', None) 
pd.options.display.max_rows = None
df[["Coordenadas"]].sort_values(by="Coordenadas").head()

In [ ]:
 df[df['Coordenadas'].str.contains(',', na=False)]

In [ ]:
# Suponiendo que tu DataFrame se llama df
df_nuevo = df.copy().dropna(subset=['Coordenadas'])
print(df.shape)
print(df_nuevo.shape)
# Crear una columna temporal con la lista de subcadenas
df_nuevo['subcadenas'] = df_nuevo['Coordenadas'].str.split(';')
print(df_nuevo.shape)
# Explode para repetir filas por cada subcadena
df_final = df_nuevo.explode('subcadenas').reset_index(drop=True)
#df_final[["NUMERO DE PROCESO", "Coordenadas","subcadenas"]]

In [ ]:
# Separando en dos columnas latitud y longitud como cadenas
#df_final[['lat_str', 'lon_str']] = df_final['subcadenas'].str.split(' ', expand=True)
df_final = df_nuevo.explode('subcadenas').reset_index(drop=True)
df_final['subcadenas'] = df_final['subcadenas'].str.replace(r'([NS])(?!\s)', r'\1 ', regex=True)
df_final['subcadenas'] = df_final['subcadenas'].str.strip().str.replace(r'\s+', ' ', regex=True)
#
df_final['subcadenas'] = df_final['subcadenas'].str.replace(r'\s+([NSEW])', r'\1', regex=True)
df_final['subcadenas'] = df_final['subcadenas'].str.replace(r'([WE]).*$', r'\1', regex=True)
df_final[['lat_str', 'lon_str']] = df_final['subcadenas'].str.split(' ', expand=True)



In [ ]:
# funcion para convertir latitud y longitud a decimal
import re

def dms_to_decimal(dms_str):
    # Extraer grados, minutos, segundos y dirección usando regex
    pattern = r'(\d+)°(\d+)\'([\d\.]+)"?([NSEW])'
    match = re.match(pattern, dms_str)
    if not match:
        return None
    
    grados, minutos, segundos, direccion = match.groups()
    decimal = float(grados) + float(minutos)/60 + float(segundos)/3600
    
    # Si es S o W, el valor es negativo
    if direccion in ['S', 'W']:
        decimal *= -1
        
    return decimal

print(dms_to_decimal('6°56\'18.0"N 73°00\'38.1"W'))

In [ ]:
df_final['latitud_dec'] = df_final['lat_str'].apply(dms_to_decimal)
df_final['longitud_dec'] = df_final['lon_str'].apply(dms_to_decimal)

In [ ]:
df_final = df_final[~(df_final.latitud_dec.isnull() | df_final.longitud_dec.isnull())].copy()


In [ ]:
# Eliminar columnas intermedias
df_final = df_final.drop(columns=['size','subcadenas','lat_str', 'lon_str'])

## Version final Coordenadas del CSV a latitud y longitud

In [ ]:
import pandas as pd
import re


# funcion para convertir latitud y longitud a decimal
def dms_to_decimal(dms_str):
    # Extraer grados, minutos, segundos y dirección usando regex
    pattern = r'(\d+)°(\d+)\'([\d\.]+)"?([NSEW])'
    match = re.match(pattern, dms_str)
    if not match:
        return None
    
    grados, minutos, segundos, direccion = match.groups()
    decimal = float(grados) + float(minutos)/60 + float(segundos)/3600
    
    # Si es S o W, el valor es negativo
    if direccion in ['S', 'W']:
        decimal *= -1
        
    return decimal


url_datos_csv = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vSfilaqDNJez3zoMqcok02nDBjZhfEwDA5PXBWfx5fOj-QJSk1C4Wb1aoQ4q0gPgQEPzgfq5tUpgsif/pub?gid=228759331&single=true&output=csv'
#Codificacion del archivo CSV. Si hay problemas con acentos, probar 'latin-1'
encoding='utf-8' 

df = pd.read_csv(url_datos_csv, encoding=encoding)
#df["size"]=df.Coordenadas.str.len()
#df = df[["NUMERO DE PROCESO", "Coordenadas","size"]]


# Suponiendo que tu DataFrame se llama df
#df_nuevo = df.copy().dropna(subset=['Coordenadas'])
df = df.dropna(subset=['Coordenadas'])

# Crear una columna temporal con la lista de subcadenas
df_nuevo['subcadenas'] = df_nuevo['Coordenadas'].str.split(';')

# Explode para repetir filas por cada subcadena
df_final = df_nuevo.explode('subcadenas').reset_index(drop=True)


# Separando en dos columnas latitud y longitud como cadenas
df_final = df_nuevo.explode('subcadenas').reset_index(drop=True)
# Adicionar un espacio despues de latitud, para los casos donde no venga separado
df_final['subcadenas'] = df_final['subcadenas'].str.replace(r'([NS])(?!\s)', r'\1 ', regex=True)

# Eliminar espacios innecesarios, inicio y fin
df_final['subcadenas'] = df_final['subcadenas'].str.strip().str.replace(r'\s+', ' ', regex=True)

# Eliminar espacios adicionales entre latitud y longitud
df_final['subcadenas'] = df_final['subcadenas'].str.replace(r'\s+([NSEW])', r'\1', regex=True)

# Eliminar cualquier caracteres despues de W o E
df_final['subcadenas'] = df_final['subcadenas'].str.replace(r'([WE]).*$', r'\1', regex=True)

# Separando la latitud y longitud en forma de cadena
df_final[['lat_str', 'lon_str']] = df_final['subcadenas'].str.split(' ', expand=True)

# Aplicando función para convertir a decimal
df_final['latitud_dec'] = df_final['lat_str'].apply(dms_to_decimal)
df_final['longitud_dec'] = df_final['lon_str'].apply(dms_to_decimal)

# Eliminar los que no tienen latitudes o longitudes válidas
df_final = df_final[~(df_final.latitud_dec.isnull() | df_final.longitud_dec.isnull())].copy()

# Eliminar columnas intermedias o innecesarias
df_final = df_final.drop(columns=['Coordenadas','subcadenas','lat_str', 'lon_str'])

In [ ]:
import pandas as pd
import re


# funcion para convertir latitud y longitud a decimal
def dms_to_decimal(dms_str):
    # Extraer grados, minutos, segundos y dirección usando regex
    pattern = r'(\d+)°(\d+)\'([\d\.]+)"?([NSEW])'
    match = re.match(pattern, dms_str)
    if not match:
        return None
    
    grados, minutos, segundos, direccion = match.groups()
    decimal = float(grados) + float(minutos)/60 + float(segundos)/3600
    
    # Si es S o W, el valor es negativo
    if direccion in ['S', 'W']:
        decimal *= -1
        
    return decimal


url_datos_csv = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vSfilaqDNJez3zoMqcok02nDBjZhfEwDA5PXBWfx5fOj-QJSk1C4Wb1aoQ4q0gPgQEPzgfq5tUpgsif/pub?gid=228759331&single=true&output=csv'
#Codificacion del archivo CSV. Si hay problemas con acentos, probar 'latin-1'
encoding='utf-8' 

df = pd.read_csv(url_datos_csv, encoding=encoding)
#df["size"]=df.Coordenadas.str.len()
#df = df[["NUMERO DE PROCESO", "Coordenadas","size"]]



#df_nuevo = df.copy().dropna(subset=['Coordenadas'])
df = df.dropna(subset=['Coordenadas'])

# Crear una columna temporal con la lista de subcadenas
df['subcadenas'] = df['Coordenadas'].str.split(';')

# Explode para repetir filas por cada subcadena
df = df.explode('subcadenas').reset_index(drop=True)


# Separando en dos columnas latitud y longitud como cadenas
df = df.explode('subcadenas').reset_index(drop=True)
# Adicionar un espacio despues de latitud, para los casos donde no venga separado
df['subcadenas'] = df['subcadenas'].str.replace(r'([NS])(?!\s)', r'\1 ', regex=True)

# Eliminar espacios innecesarios, inicio y fin
df['subcadenas'] = df['subcadenas'].str.strip().str.replace(r'\s+', ' ', regex=True)

# Eliminar espacios adicionales entre latitud y longitud
df['subcadenas'] = df['subcadenas'].str.replace(r'\s+([NSEW])', r'\1', regex=True)

# Eliminar cualquier caracteres despues de W o E
df['subcadenas'] = df['subcadenas'].str.replace(r'([WE]).*$', r'\1', regex=True)

# Separando la latitud y longitud en forma de cadena
df[['lat_str', 'lon_str']] = df['subcadenas'].str.split(' ', expand=True)

# Aplicando función para convertir a decimal
df['Latitud_decimal'] = df['lat_str'].apply(dms_to_decimal)
df['Longitud_decimal'] = df['lon_str'].apply(dms_to_decimal)

# Eliminar los que no tienen latitudes o longitudes válidas
df = df[~(df.Latitud_decimal.isnull() | df.Longitud_decimal.isnull())].copy()

# Eliminar columnas intermedias o innecesarias
df = df.drop(columns=['Coordenadas','subcadenas','lat_str', 'lon_str'])

In [149]:
df[df['NUMERO DE PROCESO'] == 'SINF-CONV-020-2025']
#df['NUMERO DE PROCESO']


,ESTADO,VIGENCIA,TIPO DE CONTRATO,SUPERVISOR DE APOYO,PERIODO,FECHA ACTA DE INICIO,FECHA PROGRAMADA TERMINACION actualizada,PLAZO o DURACION (en meses),VALOR INICIAL,VALOR ANTICIPO,VALOR FINAL,VALOR TOTAL ADICIONES $,OBJETO,OBSERVACIONES,DETALLE DE VISITAS,Alerta,NUMERO DE PROCESO,SUBDIVISION TIPO CONTRATO,TotalCobrado($),AlertaVencimiento (DiasParaFinalizar),%AvanceCobros$,Latitud_decimal,Longitud_decimal
51,Terminado-sin liquidar,2025.0,Convenio Solidario,DANIELA SERRANO LOPEZ,Primer Semestre,08/07/2025,07/10/2025,"3,0","$29.787.893,00","8.936.367,90","$29.787.893,00",$0,AUNAR ESFUERZOS ENTRE EL MUNICIPIO DE PIEDECUE...,YA TERMINARON OBRA - PENDIENTE DE ULTIMO PAGO ...,03/07/2025 - visita preliminar del punto,3,SINF-CONV-020-2025,Placa Huella,"$26.809.103,70",Pendiente por liquidación,"90,00",7.008222,-73.105667


In [ ]:
import numpy as np
np.log1p(df['TotalCobrado($)'])